In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from kiwipiepy import Kiwi
from tqdm import tqdm
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
df = pd.read_csv("/data/ephemeral/home/yujin/processed_train_4220.csv")
df = df.dropna(subset=['text'])

In [ ]:
# 각 클러스터의 데이터 개수 계산
label_counts = df['target'].value_counts().sort_index()

# 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=label_counts.index, y=label_counts.values, palette='husl')

# 그래프에 레이블 추가
for i, count in enumerate(label_counts.values):
    plt.text(i, count, str(count), ha='center', va='bottom')

# 그래프 설정
plt.title('Number of Data Points in Each Label')
plt.xlabel('label')
plt.ylabel('Number of Data Points')
plt.xticks(label_counts.index)
plt.show()

In [ ]:
kiwi = Kiwi()

In [ ]:
# 품사 분류
def part_classification(text):
    """
    텍스트의 품사를 태깅(분류)하는 함수

    Args:
        text: 데이터셋의 'text'열의 데이터

    Returns:
        raw_part_class: 각 텍스트의 토큰에 대한 품사 정보
    """
    raw_part_class = kiwi.tokenize(text, normalize_coda=False)
    return raw_part_class

In [ ]:
# 주요 품사 필터링 (명사, 동사, 형용사, 어근)
MAIN_PART = ['NNG', 'NNB', 'NNP', 'VV', 'VA', 'XR'] # 필터링할 주요 품사
VERB_ADJ_PART = ['VV', 'VA'] # 동사, 형용사

def part_filtering(text):
    """
    텍스트에서 명사, 동사, 형용사, 어근 품사만 추출하여 필터링하는 함수
    """
    tokens = kiwi.tokenize(text, normalize_coda=False)
    class_filter = [
        token.form + '다' if token.tag in VERB_ADJ_PART else token.form
        for token in tokens
        if token.tag in MAIN_PART and len(token.form) > 1
    ]
    return ' '.join(class_filter)  # 조사를 제거한 단어를 공백으로 연결

df['filtered_text'] = df['text'].apply(part_filtering)
df.head()

In [ ]:
# TF-IDF 벡터화
tfidf_vect = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)  # 유용한 feature 제한
tfidf_matrix = tfidf_vect.fit_transform(df['filtered_text'])

In [ ]:
# K-Means 클러스터링 (카테고리가 7개라고 가정)
from sklearn.cluster import KMeans

num_clusters = 7
kmeans = KMeans(n_clusters=num_clusters, max_iter=10000, random_state=42)
df['cluster'] = kmeans.fit_predict(tfidf_matrix)

In [ ]:
# 각 클러스터별로 주요 단어 추출
def get_top_keywords(tfidf_matrix, cluster_labels, n_terms=50):
    df_keywords = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vect.get_feature_names_out())
    df_keywords['cluster'] = cluster_labels
    top_keywords = {}

    for cluster in range(num_clusters):
        top_keywords[cluster] = (
            df_keywords[df_keywords['cluster'] == cluster]
            .drop('cluster', axis=1)
            .mean()
            .sort_values(ascending=False)
            .head(n_terms)
            .index.tolist()
        )
    return top_keywords


In [ ]:
# 각 클러스터별 주요 키워드 확인
top_keywords = get_top_keywords(tfidf_matrix, df['cluster'])
print("클러스터별 주요 키워드:")
for cluster, keywords in top_keywords.items():
    print(f"Cluster {cluster}: {keywords}")

In [ ]:
# 각 클러스터의 주요 단어 빈도 계산
def get_cluster_word_counts(tfidf_matrix, cluster_labels):
    """
    각 클러스터의 단어 빈도를 계산하는 함수

    Args:
        tfidf_matrix: TF-IDF로 벡터화된 텍스트 행렬
        cluster_labels: 각 텍스트의 클러스터 라벨

    Returns:
        cluster_word_counts: 각 클러스터별 단어 빈도를 저장한 리스트
    """
    # TF-IDF 행렬을 데이터프레임으로 변환하여 단어 이름을 매칭
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vect.get_feature_names_out())
    tfidf_df['cluster'] = cluster_labels
    
    # 각 클러스터별 단어 빈도 계산
    cluster_word_counts = {}
    for cluster in range(num_clusters):
        cluster_df = tfidf_df[tfidf_df['cluster'] == cluster].drop('cluster', axis=1)
        word_counts = cluster_df.sum(axis=0).sort_values(ascending=False)
        cluster_word_counts[cluster] = word_counts[word_counts > 0].index.tolist()
        
    return cluster_word_counts

# 클러스터별 단어 빈도 계산
cluster_word_counts = get_cluster_word_counts(tfidf_matrix, df['cluster'])

In [ ]:
# 클러스터 간 단어 중복 및 고유 단어 확인
def get_unique_and_common_words(cluster_word_counts):
    """
    클러스터별 고유 단어와 중복 단어를 구분하는 함수

    Args:
        cluster_word_counts: 각 클러스터별 단어 리스트

    Returns:
        unique_words: 클러스터별 고유 단어 딕셔너리
        common_words: 모든 클러스터에 중복된 단어 리스트
    """
    # 모든 클러스터의 단어들을 합집합으로 모아 공통 단어와 고유 단어 계산
    all_words = set().union(*cluster_word_counts.values())
    common_words = set(all_words)
    
    unique_words = {}
    for cluster, words in cluster_word_counts.items():
        other_clusters_words = set().union(
            *[cluster_word_counts[other] for other in cluster_word_counts if other != cluster]
        )
        unique_words[cluster] = list(set(words) - other_clusters_words)
        common_words &= set(words)
    
    return unique_words, list(common_words)

# 각 클러스터별 고유 단어와 모든 클러스터에 중복되는 단어 확인
unique_words, common_words = get_unique_and_common_words(cluster_word_counts)

In [ ]:
# 결과 출력
print("클러스터별 고유 단어:")
for cluster, words in unique_words.items():
    print(f"Cluster {cluster}: {words[:30]}")  # 상위 10개의 고유 단어만 출력

print("\n모든 클러스터에 중복된 단어:")
print(common_words[:30])  # 상위 10개의 중복 단어만 출력